In [3]:
import pandas as pd

In [4]:
url = 'https://raw.githubusercontent.com/LucaSainteCroix/teaching-resources/main/exercises-data/lda_data'

data = pd.read_csv(url, sep=",", header=None)
data.columns = ['text']
data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


In [5]:
data.shape

(1199, 1)

In [6]:
data['text'][0]

'From: gld@cunixb.cc.columbia.edu (Gary L Dare)\nSubject: Stan Fischler, 4/4\nSummary: From the Devils pregame show, prior to hosting the Penguins\nNntp-Posting-Host: cunixb.cc.columbia.edu\nReply-To: gld@cunixb.cc.columbia.edu (Gary L Dare)\nOrganization: PhDs In The Hall\nLines: 32\n\n\nAt the Lester Patrick Awards lunch, Bill Torrey mentioned that one of his\noptions next season is to be president of the Miami team, with Bob Clarke\nworking for him.  At the same dinner, Clarke said that his worst mistake\nin Philadelphia was letting Mike Keenan go -- in retrospect, almost all\nplayers came realize that Keenan knew what it took to win.  Rumours are\nnow circulating that Keenan will be back with the Flyers.\n\nNick Polano is sick of being a scapegoat for the schedule made for the\nRed Wings; After all, Bryan Murray approved it.\n\nGerry Meehan and John Muckler are worried over the Sabres\' prospects;\nAssistant Don Lever says that the Sabres have to get their share now,\nbecause a Que

In [7]:
# Traitement de texte
import re
import string
import nltk
# import spacy

# Manipulation de données
import pandas as pd
import numpy as np

# Vectorisation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Modèles ML
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# Pour plus tard : deep learning
# import tensorflow as tf
# from tensorflow import keras

# Téléchargement des ressources nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\marin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## preprocess

In [8]:
def preprocess_text(text):
    # 1. Mettre en minuscules
    text = text.lower()
    
    # 2. Enlever la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 3. Tokenisation
    tokens = word_tokenize(text)
    
    # 4. Suppression des stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Lemmatisation
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return " ".join(tokens)


In [9]:
sample_text = "The BBC reported that the economy is slowing down, but some sectors remain strong!"
print("Avant :", sample_text)
print("Après :", preprocess_text(sample_text))


Avant : The BBC reported that the economy is slowing down, but some sectors remain strong!
Après : bbc reported economy slowing sector remain strong


In [10]:
# Mini dataset fictif pour tester le pipeline
data = {
    'text': [
        "The match was incredible, our team won easily!",
        "The prime minister gave a speech about the economy.",
        "A new technology has been released by Apple.",
        "Manchester United signed a new player this week.",
        "The government announced new tax policies.",
        "Scientists discovered a new species of fish."
    ],
    'category': ['sport', 'politics', 'tech', 'sport', 'politics', 'tech']
}

df = pd.DataFrame(data)
df['clean_text'] = df['text'].apply(preprocess_text)
df.head()


,text,category,clean_text
0,"The match was incredible, our team won easily!",sport,match incredible team easily
1,The prime minister gave a speech about the eco...,politics,prime minister gave speech economy
2,A new technology has been released by Apple.,tech,new technology released apple
3,Manchester United signed a new player this week.,sport,manchester united signed new player week
4,The government announced new tax policies.,politics,government announced new tax policy


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialisation du vectoriseur
vectorizer = TfidfVectorizer()

# Application sur les textes nettoyés
X = vectorizer.fit_transform(df['clean_text'])

# Affichage du vocabulaire appris
print("Nombre de mots uniques :", len(vectorizer.get_feature_names_out()))


Nombre de mots uniques : 26


## Modèle LDA

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Cibles (catégories)
y = df['category']

# Séparation train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Modèle
model = LogisticRegression()
model.fit(X_train, y_train)

# Prédiction
y_pred = model.predict(X_test)

# Évaluation
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    politics       0.00      0.00      0.00       1.0
       sport       0.00      0.00      0.00       1.0
        tech       0.00      0.00      0.00       0.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



d:\Users\marin\Desktop\NLP\NPL-1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Users\marin\Desktop\NLP\NPL-1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Users\marin\Desktop\NLP\NPL-1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Use

## Chat Bot

In [19]:
import joblib
import os

# Crée le dossier app/ si besoin
os.makedirs("../app", exist_ok=True)

In [21]:
# joblib.dump(model, "../app/model.joblib")
# joblib.dump(vectorizer, "../app/vectorizer.joblib")

joblib.dump(model, "model.joblib")
joblib.dump(vectorizer, "vectorizer.joblib")

['vectorizer.joblib']

## Fonction de résumé simple (baseline)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [ ]:
texte = """The economy is showing signs of slowing down after record inflation.
However, some sectors like technology remain strong.
The prime minister addressed this in a recent press conference.
Analysts believe the growth will stabilize next year.
Tax reforms are also being considered to support businesses."""

print(summarize_text(texte))


The economy is showing signs of slowing down after record inflation. Tax reforms are also being considered to support businesses.


## integration dans le chatbot